In [1]:
# Importing file
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
def is_number(n):
    if str(n).lower() == 'nan':
        return False
    try:
        float(n)
    except ValueError:
        return False
    return True

In [3]:
all_data = pd.read_excel("19965 - 19997_Dist 3_Department WB_DateMod_20180926 101958.xlsx", sheetname='Audit')
all_data.iloc[:,3] = all_data.iloc[:,3].astype(str)
all_data.iloc[:,1] = all_data.iloc[:, 1].astype(str)
# Removing the row that doens't start with $
all_data = all_data[all_data.iloc[:, 0].str.contains("^\$")]
all_data = all_data[all_data.iloc[:, 3].str.contains("^D.*")]
all_data = all_data[all_data.iloc[:, 1].apply(lambda x: is_number(x))]
all_data["hash"] = all_data.iloc[:, 0].str.cat(all_data.iloc[:, 3])
all_data['hash'] = all_data["hash"].apply(lambda x: hash(x))
all_data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,hash
837,$U$18,324,2018-08-13 23:14:47,D(1),C:\Users\costnera\Desktop,QASP v1.1 workbook_2018-08-06_Department.xlsb,MATERIALTESTING.LOCAL,COSTNERA,\\DC,costnera,Alex Costner,19965 - 19997,NaN,NaN,NaN,NaN,NaN,-2798281106177065461
840,$U$20,22444,2018-08-13 23:15:02,D(1),C:\Users\costnera\Desktop,QASP v1.1 workbook_2018-08-06_Department.xlsb,MATERIALTESTING.LOCAL,COSTNERA,\\DC,costnera,Alex Costner,19965 - 19997,NaN,NaN,NaN,NaN,NaN,7459668802582935615
842,$AU$20,94,2018-08-13 23:15:09,D(1),C:\Users\costnera\Desktop,QASP v1.1 workbook_2018-08-06_Department.xlsb,MATERIALTESTING.LOCAL,COSTNERA,\\DC,costnera,Alex Costner,19965 - 19997,NaN,NaN,NaN,NaN,NaN,5484336246250714980
846,$U$25,22444,2018-08-13 23:17:17,D(1),C:\Users\costnera\Desktop,QASP v1.1 workbook_2018-08-06_Department.xlsb,MATERIALTESTING.LOCAL,COSTNERA,\\DC,costnera,Alex Costner,19965 - 19997,NaN,NaN,NaN,NaN,NaN,9204937580953207336
850,$BE$25,301,2018-08-13 23:17:56,D(1),C:\Users\costnera\Desktop,QASP v1.1 workbook_2018-08-06_Department.xlsb,MATERIALTESTING.LOCAL,COSTNERA,\\DC,costnera,Alex Costner,19965 - 19997,NaN,NaN,NaN,NaN,NaN,297489032802840460


In [4]:
# Getting the repeated one only
repeated = pd.read_excel("Result_dep_19965.xlsx", sheetname="Sheet1")
repeated = repeated.drop(["Var_time_gap", "Var_effect_type"], axis=1)
repeated["hash"] = repeated.Cell.str.cat(repeated["Sample"])
repeated['hash'] = repeated["hash"].apply(lambda x: hash(x))

In [5]:
# non_repeated_hash = list(set(all_data.hash).difference(set(repeated.hash)))
repeated_hash = set(repeated.hash)
non_repeated_data = []
for index, row in all_data.iterrows():
    if row["hash"] not in repeated_hash:
#         The repeated dataset doesn't have this value so we will just add it to repeated datacell as non repeat
        single_non_repeated_data = [row[3], row[0], row[1], row[2], "non-repeated", row["hash"]]
        non_repeated_data.append(single_non_repeated_data)
        
non_repeated = pd.DataFrame(non_repeated_data, columns = repeated.columns.values)
final_df = pd.concat([repeated, non_repeated])
final_df.head()

,Sample,Cell,Value,Time,Var_error,hash
0,D-DENS(1),$AC$37,142.5,2018-08-17 06:21:40,Fraud,-4913598893637652684
1,D-DENS(1),$AC$37,138.4,2018-08-21 12:52:51,Fraud,-4913598893637652684
2,D-DENS(2),$AC$37,140.7,2018-08-18 05:13:41,Fraud,-7686383880273735245
3,D-DENS(2),$AC$37,142.5,2018-08-21 12:55:22,Fraud,-7686383880273735245
4,D-DENS(3),$AC$37,138.5,2018-08-20 06:00:17,Fraud,4234954773531695530


In [6]:
final_df.to_csv("Dep_Typo_Fraud_NonRepeat_19965.csv",index=True,header = True)

In [7]:
data_Range = pd.read_excel("Dep_value_Range_19965.xlsx", sheetname='Sheet1')
data_Range.head()

,Var1_1,Var1_2,Var1_3
0,$U$32,2122.8,2150.5
1,$U$33,3627.9,3823.3
2,$U$37,2224.6,2340.2
3,$U$38,1337.2,1355.7
4,$Z$32,2122.7,2156.6


In [8]:
first_typo_first_fraud = pd.DataFrame(columns=set(final_df.Cell), index=set(final_df.Sample))
first_typo_last_fraud = pd.DataFrame(columns=set(final_df.Cell), index=set(final_df.Sample))
last_typo_first_fraud = pd.DataFrame(columns=set(final_df.Cell), index=set(final_df.Sample))
last_typo_last_fraud = pd.DataFrame(columns=set(final_df.Cell), index=set(final_df.Sample))
already_inserted = []
for index, row in final_df.iterrows():
    if row["hash"] in already_inserted:
        continue
    if row["Var_error"] == "non-repeated":
        first_typo_first_fraud[row["Cell"]][row["Sample"]] = float(row["Value"])
        first_typo_last_fraud[row["Cell"]][row["Sample"]] = float(row["Value"])
        last_typo_first_fraud[row["Cell"]][row["Sample"]] = float(row["Value"])
        last_typo_last_fraud[row["Cell"]][row["Sample"]] = float(row["Value"])
    elif row["Var_error"] == "Typo":
        # We will take the last one
        # Get all typo's entry of this test
        all_typo = final_df[final_df["hash"] == row["hash"]]
        all_typo["Value"] = all_typo["Value"].apply(pd.to_numeric, errors='coerce')
        all_typo = all_typo.dropna(subset=['Value'])
        all_typo.sort_values("Time", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')
                # all_typo
        for i in range(all_typo.shape[0]):
            if any(all_typo.iloc[i]["Cell"] == data_Range['Var1_1']) and all_typo.iloc[i]["Value"] >= (data_Range[all_typo.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_2']).values[0] and all_typo.iloc[i]["Value"] <= (data_Range[all_typo.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_3']).values[0]:
                first_typo_first_fraud[row["Cell"]][row["Sample"]] = float(all_typo.iloc[i]["Value"])
                first_typo_last_fraud[row["Cell"]][row["Sample"]] = float(all_typo.iloc[i]["Value"])
                break
        for i in reversed(range(all_typo.shape[0])):
            if any(all_typo.iloc[i]["Cell"] == data_Range['Var1_1']) and all_typo.iloc[i]["Value"] >= (data_Range[all_typo.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_2']).values[0] and all_typo.iloc[i]["Value"] <= (data_Range[all_typo.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_3']).values[0]:
                last_typo_first_fraud[row["Cell"]][row["Sample"]] = float(all_typo.iloc[i]["Value"])
                last_typo_last_fraud[row["Cell"]][row["Sample"]] = float(all_typo.iloc[i]["Value"])
                break
#         else:
#             last_typo_first_fraud[row["Cell"]][row["Sample"]] = float(all_typo.iloc[-2]["Value"])
#             last_typo_last_fraud[row["Cell"]][row["Sample"]] = float(all_typo.iloc[-2]["Value"])
            
    elif row["Var_error"] == "Fraud":
        all_fraud = final_df[final_df["hash"] == row["hash"]]
        all_fraud["Value"] = all_fraud["Value"].apply(pd.to_numeric, errors='coerce')
        all_fraud = all_fraud.dropna(subset=['Value'])
        all_fraud.sort_values("Time", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')
#         if all_fraud.shape[0] == 0:
#             continue
        for i in range(all_fraud.shape[0]):
            if any(all_fraud.iloc[i]["Cell"] == data_Range['Var1_1']) and all_fraud.iloc[i]["Value"] >= (data_Range[all_fraud.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_2']).values[0] and all_fraud.iloc[i]["Value"] <= (data_Range[all_fraud.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_3']).values[0]:
                first_typo_first_fraud[row["Cell"]][row["Sample"]] = float(all_fraud.iloc[i]["Value"])
                last_typo_first_fraud[row["Cell"]][row["Sample"]] = float(all_fraud.iloc[i]["Value"])
                break
        for i in reversed(range(all_fraud.shape[0])):
            if any(all_fraud.iloc[i]["Cell"] == data_Range['Var1_1']) and all_fraud.iloc[i]["Value"] >= (data_Range[all_fraud.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_2']).values[0] and all_fraud.iloc[i]["Value"] <= (data_Range[all_fraud.iloc[i]["Cell"] == data_Range['Var1_1']]['Var1_3']).values[0]:
                last_typo_last_fraud[row["Cell"]][row["Sample"]] = float(all_fraud.iloc[i]["Value"])
                first_typo_last_fraud[row["Cell"]][row["Sample"]] = float(all_fraud.iloc[i]["Value"])
                break
    else:
        print("Print Nothing")
        
    already_inserted.append(row["hash"])


In [9]:
first_typo_first_fraud = first_typo_first_fraud.astype(float)
first_typo_last_fraud = first_typo_last_fraud.astype(float)
last_typo_first_fraud = last_typo_first_fraud.astype(float)
last_typo_last_fraud = last_typo_last_fraud.astype(float)


last_typo_first_fraud.to_csv("last_typo_first_fraud_dep_19965_2.csv")
last_typo_last_fraud.to_csv("last_typo_last_fraud_dep_19965_2.csv")